In [ ]:
import nltk
nltk.download('stopwords')

In [2]:
from googleapiclient import discovery
from httplib2 import Http
import oauth2client
from oauth2client import file, client, tools
obj = lambda: None
lmao = {"auth_host_name":'localhost', 'noauth_local_webserver':'store_true', 'auth_host_port':[8080, 8090], 'logging_level':'ERROR'}
for k, v in lmao.items():
    setattr(obj, k, v)
    
# authorization boilerplate code
SCOPES = 'https://www.googleapis.com/auth/drive.readonly'
store = file.Storage('token.json')
creds = store.get()
# The following will give you a link if token.json does not exist, the link allows the user to give this app permission
if not creds or creds.invalid:
    flow = client.flow_from_clientsecrets('client_id.json', SCOPES)
    creds = tools.run_flow(flow, store, obj)

In [3]:
import io
from googleapiclient.http import MediaIoBaseDownload
DRIVE = discovery.build('drive', 'v3', http=creds.authorize(Http()))
# if you get the shareable link, the link contains this id, replace the file_id below
file_id = '1Vf8aByqXQwTVmxxs5jKenvAXaxSF6TsH'
request = DRIVE.files().get_media(fileId=file_id)
# replace the filename and extension in the first field below
fh = io.FileIO('wiki-news-300d-1M.vec', mode='w')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))

Download 4%.
Download 9%.
Download 13%.
Download 18%.
Download 23%.
Download 27%.
Download 32%.
Download 37%.
Download 41%.
Download 46%.
Download 51%.
Download 55%.
Download 60%.
Download 64%.
Download 69%.
Download 74%.
Download 78%.
Download 83%.
Download 88%.
Download 92%.
Download 97%.
Download 100%.


In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import keras
from keras import optimizers
from keras import backend as K
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from keras.utils import plot_model
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer 
import os, re, csv, math, codecs

sns.set_style("whitegrid")
np.random.seed(0)

MAX_NB_WORDS = 100000
tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))
stop_words.update(['.', ',', '"', "'", ':', ';', '(', ')', '[', ']', '{', '}'])

Using TensorFlow backend.


In [2]:
embeddings_index = {}
f = codecs.open('./wiki-news-300d-1M.vec', encoding='utf-8')
for line in tqdm(f):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('found %s word vectors' % len(embeddings_index))

999995it [02:13, 7487.59it/s]


found 999995 word vectors


In [3]:
def splitIntoStem(message):
    return [removeNumeric(stripEmoji(singleCharacterRemove(removePunctuation
                                                           (removeHyperlinks
                                                            (removeHashtags
                                                             (removeUsernames
                                                              (stemWord(word)))))))) for word in message.split()]
def stemWord(tweet):
    return tweet.lower()

def removeUsernames(tweet):
    return re.sub('@[^\s]+', '', tweet)

def removeHashtags(tweet):
    return re.sub(r'#[^\s]+', '', tweet)

def removeHyperlinks(tweet):
    return re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', tweet)

def removeNumeric(value):
    blist2 = [item for item in value if not item.isdigit()]
    blist3 = "".join(blist2)
    return blist3

def removePunctuation(tweet):

    return re.sub(r'[^\w\s]','',tweet)

def singleCharacterRemove(tweet):
    return re.sub(r'(?:^| )\w(?:$| )', ' ', tweet)

def stripEmoji(text):

    RE_EMOJI = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)
    return RE_EMOJI.sub(r'', text)

In [4]:
tweets_df = pd.read_csv('./tweets.csv', sep=',', header=0)
tweet_list = tweets_df['Tweet'].tolist()
label_list = tweets_df['Label'].tolist()
one_hot_labels = keras.utils.to_categorical(label_list, num_classes=4)

In [5]:
processed_list = []
for sentence in tqdm(tweet_list):
    tokens = " ".join(splitIntoStem(sentence)).split()
    filtered = [word for word in tokens if word not in stop_words]
    processed_list.append(" ".join(filtered))

100%|████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 3348.66it/s]


In [6]:
df = pd.DataFrame()
df["tweet"] = processed_list
df['doc_len'] = df['tweet'].apply(lambda words: len(words.split(" ")))
max_seq_len = np.round(df['doc_len'].mean() + df['doc_len'].std()).astype(int)
max_seq_len

35

In [107]:
x_train, x_test, y_train, y_test = train_test_split(processed_list, one_hot_labels, test_size=0.35)

In [108]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train + x_test)
word_seq_train = tokenizer.texts_to_sequences(x_train)
word_seq_test = tokenizer.texts_to_sequences(x_test)
word_index = tokenizer.word_index
print("dictionary size: ", len(word_index))

dictionary size:  395


In [109]:
num_tokens = [len(tokens) for tokens in word_seq_train + word_seq_test]
num_tokens = np.array(num_tokens)
num_tokens

array([ 12,  14,  12,   5,   6, 112,   7,   5,   9,   5,  14,   9,   7,
         7,   6,   8,   7,  55,  13,  15,   4,  14,   4,  13,  17,   9,
         9,  13,  14,  19])

In [110]:
word_seq_train = sequence.pad_sequences(word_seq_train, maxlen=max_seq_len)
word_seq_test = sequence.pad_sequences(word_seq_test, maxlen=max_seq_len)

In [111]:
word_seq_test = word_seq_test[:-1]
y_test = y_test[:-1]

In [112]:
#training params
batch_size = 4
num_epochs = 8 

#model parameters
num_filters = 64 
embed_dim = 300 
weight_decay = 1e-4

In [113]:
words_not_found = []
nb_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_dim))
for word, i in word_index.items():
    if i >= nb_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if (embedding_vector is not None) and len(embedding_vector) > 0:
        embedding_matrix[i] = embedding_vector
    else:
        words_not_found.append(word)
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
print("sample words not found: ", np.random.choice(words_not_found, 10))

number of null word embeddings: 23
sample words not found:  ['bookershortlisted' 'jehovahs' 'neeson' 'pridechoose' 'madeley'
 'yagerûªs' 'madeley' 'jarre' 'diaghilev' 'leahycrapo']


In [114]:
model = Sequential()
model.add(Embedding(num_words, embed_dim, weights=[embedding_matrix], input_length=max_seq_len, trainable=False))
model.add(Dropout(0.2))
model.add(Conv1D(num_filters, 5, padding='same', activation='relu', strides=1))
model.add(MaxPooling1D(2))
model.add(Conv1D(num_filters, 5, activation='relu', padding='same'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 35, 300)           118500    
_________________________________________________________________
dropout_11 (Dropout)         (None, 35, 300)           0         
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 35, 64)            96064     
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 17, 64)            0         
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 17, 64)            20544     
_________________________________________________________________
global_max_pooling1d_6 (Glob (None, 64)                0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 64)                0         
__________

In [137]:
model.fit(word_seq_train, y_train)

Epoch 1/1
19/19 [==============================] - 0s 1ms/step - loss: 0.6308 - acc: 0.8947


In [138]:
model.evaluate(word_seq_test, y_test)

10/10 [==============================] - 0s 23ms/step


[0.9873528480529785, 0.699999988079071]